<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 文档标记

In [ ]:
from videos.walkthroughs import walkthrough_43 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何从长文本中提取并标记数据来扩展生成结构化数据的技能。

---

## 目标

完成这个 notebook 后，您将能够：

- 构建代表其它 Pydantic 类集合的 Pydantic 类。
- 对长文本进行提取和标记。

---

## 导入

In [ ]:
from typing import List
from pprint import pprint

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 文档标记

您已经了解如何创建 Pydantic 规范以生成结构化数据了，现在就可以轻松地将这项技能扩展到从长文本中提取和标记数据。

为了学习这个技巧，假设我们想要提取文本中提到的任何一种水果的名称。像之前一样，先定义一个数据模式，然后实例化一个解析器，结合使用相应格式的提示词指令，就能从提示词和 LLM 推断的内容中解析出结构化数据。

In [ ]:
class Fruit(BaseModel):
    """The name of a piece of fruit."""

    name: str = Field(description="The name of the piece of fruit")

In [ ]:
parser = JsonOutputParser(pydantic_object=Fruit)

In [ ]:
format_instructions = parser.get_format_instructions()

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        "Generate JSON about the user input according to the provided format instructions.\n" +
        "Input: {input}\n" +
        "Format instructions {format_instructions}")
    )
])

In [ ]:
template_with_format_instructions = template.partial(format_instructions=format_instructions)

In [ ]:
chain = template_with_format_instructions | llm | parser

现在要做的事情和之前的 notebook 略有不同。我们不提供单个用于转换为结构化数据的实体，而是提供自由格式的文本。

考虑到以下语句的简单性，毫无疑问，我们的链能很好地识别并捕获提到的那一种水果。

In [ ]:
chain.invoke({"input": "An apple fell from the tree."})

---

## 结构化数据列表

从自由格式文本中提取和标记多个数据实体时，还缺少一个特性，就是提取一个**列表**而不是只是某个数据类型的唯一实体。

这时可以使用 Pydantic 结合 Python 的 `typing.List`，相当简单：创建一个新的 Pydantic 类，并附上有用的 docstring，带上一个包含其它 Pydantic 类的 `List`。

In [ ]:
from typing import List

In [ ]:
class Fruits(BaseModel):
    """The names of fruits"""
    fruits: List[Fruit]

现在，我们可以利用带有列表的 `Fruits` 类来构建解析器和链。

In [ ]:
parser = JsonOutputParser(pydantic_object=Fruits)

In [ ]:
format_instructions = parser.get_format_instructions()

In [ ]:
template_with_format_instructions = template.partial(format_instructions=format_instructions)

In [ ]:
chain = template_with_format_instructions | llm | parser

现在当我们传入一段包含多种水果的较长文本时，可以看到能提取和标记所有这些水果了。

In [ ]:
chain.invoke({"input": "An apple fell from the tree. It hit the ground right next to a banana peel."})

---

## 练习：为阿波罗故事进行文档标记

以下是阿波罗 11 登陆的描述。本练习的目标是从描述中提取和标记几个实体。

具体来说，您应该提取和标记以下内容：
- 关于整个登陆的细节，包括
    - 描述中提到的任何机组成员的列表。对于每位机组成员，您应获取他们的：
        - 姓名
        - 任务中的角色
    - 描述中提到的任何航天器的部件和模块的列表。对于提取的航天器部件，您应获取其：
        - 名称
        - 所属的具体部件或模块
    - 描述中提到的任何重要引用的列表。对于每个重要引用，您应提取和标记：
        - 引用本身
        - 引用的发言者的姓名

如果您想，可以直接开始。当然也可以展开下面的*指导*部分。

In [ ]:
apollo_story = """
On July 20, 1969, Apollo 11, the first manned mission to land on the Moon, successfully touched down in the Sea of Tranquility. \
The crew consisted of Neil Armstrong, who served as the mission commander, \
Edwin 'Buzz' Aldrin, the lunar module pilot, and Michael Collins, the command module pilot.

The spacecraft consisted of two main parts: the command module Columbia and the lunar module Eagle. \
As Armstrong stepped onto the lunar surface, he famously declared, "That's one small step for man, one giant leap for mankind."

Buzz Aldrin also descended onto the Moon's surface, where he and Armstrong conducted experiments and collected samples. \
Michael Collins remained in lunar orbit aboard Columbia, ensuring the successful return of his fellow astronauts.

The mission was a pivotal moment in space exploration and remains a significant achievement in human history.
"""

### 您的代码

---

## 指导

### 定义机组成员详细信息

按照上述指南，创建一个表示给定机组成员详细信息的类。

如果您卡住了，欢迎查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
class CrewMember(BaseModel):
    """Details of a crew member"""
    name: str = Field(description="Name of the crew member")
    role: str = Field(description="Role of the crew member in the mission")

### 定义航天器详细信息

按照上述指南，创建一个表示描述中提到的航天器详细信息的类。

如果您卡住了，欢迎查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
class SpacecraftDetail(BaseModel):
    """Details of the spacecraft"""
    name: str = Field(description="Name of the spacecraft")
    part: str = Field(description="Specific part or module of the spacecraft")

### 定义重要引用

按照上述指南，创建一个表示描述中所有重要引用详细信息的类。

如果您卡住了，欢迎查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
class SignificantQuote(BaseModel):
    """Details of a significant quote"""
    quote: str = Field(description="The quote")
    speaker: str = Field(description="Name of the person who said the quote")

### 定义整个着陆的综合详情

创建一个类，包含阿波罗 11 任务的综合详情。它应该包含您上述创建的其它 3 个类的列表。

如果您卡住了，欢迎查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
class Apollo11Details(BaseModel):
    """Combined details of the Apollo 11 mission"""
    crew_members: List[CrewMember]
    spacecraft_details: List[SpacecraftDetail]
    significant_quotes: List[SignificantQuote]

### 创建提取链

在所有数据类都定义好之后，是时候创建一个链，包括将 `JsonOutputParser` 与我们的 LLM 实例结合，执行实际的提取和标记工作了。

如果您卡住了，可以查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
parser = JsonOutputParser(pydantic_object=Apollo11Details)

format_instructions = parser.get_format_instructions()

template_with_format_instructions = template.partial(format_instructions=format_instructions)

chain = template_with_format_instructions | llm | parser

### 调用提取链

现在，对 apollo_story 调用链吧。

如果您卡住了，可以查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
apollo_details = chain.invoke({"input": apollo_story})

In [ ]:
pprint(apollo_details)

---

## 总结

本 notebook 结束了关于结构化数据生成的部分，我们希望您能认识到这是一种功能强大的工具，有很多应用。

与生成结构化数据的能力相关，LLM 可以生成结构化数据，指示何时以及如何调用非 LLM 功能。我们称这种技术为工具使用（tool use），接下来的部分您将学习如何创建工具，并通过智能体将其与 LLM 的交互结合起来。